# 导入环境

In [1]:
from datasets import Dataset
import pandas as pd
from transformers import AutoTokenizer, AutoModelForCausalLM, DataCollatorForSeq2Seq, TrainingArguments, Trainer, GenerationConfig

2025-04-12 01:12:04.835840: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-04-12 01:12:04.846921: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1744391524.860609    9664 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1744391524.864709    9664 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1744391524.874908    9664 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

In [2]:
# 将JSON文件转换为CSV文件
df = pd.read_json('./dataset/mes_data.json')
ds = Dataset.from_pandas(df)

In [3]:
ds[:3]

{'instruction': ['请分析设备E404的E305报警可能的原因及优化建议。',
  '工单编号W1001的实际完工时间比计划晚2小时，请分析原因并提出优化建议。',
  '产线X上的产品Y610产量未达标，请分析原因。'],
 'input': ['报警代码：E305（电机过载）；发生时间：08:30；持续时间：10分钟。\n最近一次维护时间为3天前。',
  '产品：Y610；计划开始：08:00；实际开始：09:30；\n设备E202在9:15出现E507报警。',
  '产品型号：Y610；标准产量：1000；实际产量：720；\n工艺时间偏长，设备E202在中途发生异常。'],
 'output': ['E404发生电机过载可能由于传感器老化或现场温度波动剧烈，建议检查传感器连接并调整工艺参数。',
  '工单延迟原因可能包括：前序工序延误、设备E202出现冷却系统异常报警。建议设置设备预热并安排提前开工。',
  '产量未达标原因包括设备效率低、工序衔接不顺畅。建议优化排产顺序并增加设备检查频次。']}

# 处理数据集

In [4]:
tokenizer = AutoTokenizer.from_pretrained('./autodl-tmp/qwen/Qwen2___5-7B-Instruct/', use_fast=False, trust_remote_code=True)
tokenizer

Qwen2Tokenizer(name_or_path='./autodl-tmp/qwen/Qwen2___5-7B-Instruct/', vocab_size=151643, model_max_length=131072, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'eos_token': '<|im_end|>', 'pad_token': '<|endoftext|>', 'additional_special_tokens': ['<|im_start|>', '<|im_end|>', '<|object_ref_start|>', '<|object_ref_end|>', '<|box_start|>', '<|box_end|>', '<|quad_start|>', '<|quad_end|>', '<|vision_start|>', '<|vision_end|>', '<|vision_pad|>', '<|image_pad|>', '<|video_pad|>']}, clean_up_tokenization_spaces=False),  added_tokens_decoder={
	151643: AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	151644: AddedToken("<|im_start|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	151645: AddedToken("<|im_end|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	151646: AddedToken("<|object_ref_start|>", rstrip=False, lstrip=False, singl

In [5]:
def process_func(example):
    MAX_LENGTH = 384    # Llama分词器会将一个中文字切分为多个token，因此需要放开一些最大长度，保证数据的完整性
    input_ids, attention_mask, labels = [], [], []
    instruction = tokenizer(f"<|im_start|>system\n现在你要扮演一个精通预测性维护的老专家的角色<|im_end|>\n<|im_start|>user\n{example['instruction'] + example['input']}<|im_end|>\n<|im_start|>assistant\n", add_special_tokens=False)  # add_special_tokens 不在开头加 special_tokens
    response = tokenizer(f"{example['output']}", add_special_tokens=False)
    input_ids = instruction["input_ids"] + response["input_ids"] + [tokenizer.pad_token_id]
    attention_mask = instruction["attention_mask"] + response["attention_mask"] + [1]  # 因为eos token咱们也是要关注的所以 补充为1
    labels = [-100] * len(instruction["input_ids"]) + response["input_ids"] + [tokenizer.pad_token_id]  
    if len(input_ids) > MAX_LENGTH:  # 做一个截断
        input_ids = input_ids[:MAX_LENGTH]
        attention_mask = attention_mask[:MAX_LENGTH]
        labels = labels[:MAX_LENGTH]
    return {
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "labels": labels
    }

In [6]:
tokenized_id = ds.map(process_func, remove_columns=ds.column_names)
tokenized_id

Map:   0%|          | 0/50 [00:00<?, ? examples/s]

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 50
})

In [7]:
tokenized_id[0]['input_ids']

[151644,
 8948,
 198,
 99601,
 105182,
 102889,
 46944,
 114806,
 104538,
 33071,
 101999,
 105759,
 101057,
 107201,
 151645,
 198,
 151644,
 872,
 198,
 14880,
 101042,
 101044,
 36,
 19,
 15,
 19,
 9370,
 36,
 18,
 15,
 20,
 106125,
 87267,
 104249,
 81217,
 103983,
 101898,
 1773,
 106125,
 46100,
 5122,
 36,
 18,
 15,
 20,
 9909,
 107114,
 38182,
 27366,
 7552,
 24968,
 99726,
 20450,
 5122,
 15,
 23,
 25,
 18,
 15,
 24968,
 100652,
 20450,
 5122,
 16,
 15,
 83031,
 8997,
 104044,
 99796,
 101999,
 112586,
 18,
 35727,
 24562,
 1773,
 151645,
 198,
 151644,
 77091,
 198,
 36,
 19,
 15,
 19,
 99726,
 107114,
 38182,
 27366,
 87267,
 101887,
 107779,
 109964,
 57191,
 100647,
 104273,
 105524,
 110963,
 3837,
 101898,
 101071,
 107779,
 64064,
 62926,
 101921,
 101189,
 32665,
 1773,
 151643]

In [8]:
tokenizer.decode(tokenized_id[0]['input_ids'])

'<|im_start|>system\n现在你要扮演一个精通预测性维护的老专家的角色<|im_end|>\n<|im_start|>user\n请分析设备E404的E305报警可能的原因及优化建议。报警代码：E305（电机过载）；发生时间：08:30；持续时间：10分钟。\n最近一次维护时间为3天前。<|im_end|>\n<|im_start|>assistant\nE404发生电机过载可能由于传感器老化或现场温度波动剧烈，建议检查传感器连接并调整工艺参数。<|endoftext|>'

In [9]:
tokenizer.decode(list(filter(lambda x: x != -100, tokenized_id[1]["labels"])))

'工单延迟原因可能包括：前序工序延误、设备E202出现冷却系统异常报警。建议设置设备预热并安排提前开工。<|endoftext|>'

# 创建模型

In [10]:
import torch

model = AutoModelForCausalLM.from_pretrained('./autodl-tmp/qwen/Qwen2___5-7B-Instruct/', device_map="auto",torch_dtype=torch.bfloat16)
model

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(152064, 3584)
    (layers): ModuleList(
      (0-27): 28 x Qwen2DecoderLayer(
        (self_attn): Qwen2SdpaAttention(
          (q_proj): Linear(in_features=3584, out_features=3584, bias=True)
          (k_proj): Linear(in_features=3584, out_features=512, bias=True)
          (v_proj): Linear(in_features=3584, out_features=512, bias=True)
          (o_proj): Linear(in_features=3584, out_features=3584, bias=False)
          (rotary_emb): Qwen2RotaryEmbedding()
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear(in_features=3584, out_features=18944, bias=False)
          (up_proj): Linear(in_features=3584, out_features=18944, bias=False)
          (down_proj): Linear(in_features=18944, out_features=3584, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen2RMSNorm((3584,), eps=1e-06)
        (post_attention_layernorm): Qwen2RMSNorm((3584,), eps=1e-06)
      )
    )
    (norm):

In [11]:
model.enable_input_require_grads() # 开启梯度检查点时，要执行该方法

In [12]:
model.dtype

torch.bfloat16

# lora 

In [13]:
from peft import LoraConfig, TaskType, get_peft_model

config = LoraConfig(
    task_type=TaskType.CAUSAL_LM, 
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    inference_mode=False, # 训练模式
    r=8, # Lora 秩
    lora_alpha=32, # Lora alaph，具体作用参见 Lora 原理
    lora_dropout=0.1# Dropout 比例
)
config

LoraConfig(peft_type=<PeftType.LORA: 'LORA'>, auto_mapping=None, base_model_name_or_path=None, revision=None, task_type=<TaskType.CAUSAL_LM: 'CAUSAL_LM'>, inference_mode=False, r=8, target_modules={'gate_proj', 'down_proj', 'o_proj', 'k_proj', 'q_proj', 'v_proj', 'up_proj'}, lora_alpha=32, lora_dropout=0.1, fan_in_fan_out=False, bias='none', use_rslora=False, modules_to_save=None, init_lora_weights=True, layers_to_transform=None, layers_pattern=None, rank_pattern={}, alpha_pattern={}, megatron_config=None, megatron_core='megatron.core', loftq_config={}, use_dora=False, layer_replication=None)

In [14]:
model = get_peft_model(model, config)
config

LoraConfig(peft_type=<PeftType.LORA: 'LORA'>, auto_mapping=None, base_model_name_or_path='./autodl-tmp/qwen/Qwen2___5-7B-Instruct/', revision=None, task_type=<TaskType.CAUSAL_LM: 'CAUSAL_LM'>, inference_mode=False, r=8, target_modules={'gate_proj', 'down_proj', 'o_proj', 'k_proj', 'q_proj', 'v_proj', 'up_proj'}, lora_alpha=32, lora_dropout=0.1, fan_in_fan_out=False, bias='none', use_rslora=False, modules_to_save=None, init_lora_weights=True, layers_to_transform=None, layers_pattern=None, rank_pattern={}, alpha_pattern={}, megatron_config=None, megatron_core='megatron.core', loftq_config={}, use_dora=False, layer_replication=None)

In [15]:
model.print_trainable_parameters()

trainable params: 20,185,088 || all params: 7,635,801,600 || trainable%: 0.2643


# 配置训练参数

In [16]:
args = TrainingArguments(
    output_dir="./output/mes_lora",
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    logging_steps=10,
    logging_dir="./output/mes_lora/logs",
    log_level="info",
    num_train_epochs=50,
    save_steps=100, 
    learning_rate=1e-4,
    save_on_each_node=True,
    gradient_checkpointing=True
)

In [17]:
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_id,
    data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer, padding=True),
)

In [18]:
trainer.train()

***** Running training *****
  Num examples = 50
  Num Epochs = 50
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 4
  Total optimization steps = 150
  Number of trainable parameters = 20,185,088
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
/root/jupyter_gpu/lib/python3.10/site-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss
10,2.570700
20,0.162700
30,0.003500
40,0.001100
50,0.000200
60,0.000100
70,0.000000
80,0.000000
90,0.000000
100,0.000000


Saving model checkpoint to ./output/mes_lora/checkpoint-100
/root/jupyter_gpu/lib/python3.10/site-packages/peft/utils/save_and_load.py:195: UserWarning: Could not find a config file in ./autodl-tmp/qwen/Qwen2___5-7B-Instruct/ - will assume that the vocabulary was not modified.
  warnings.warn(
/root/jupyter_gpu/lib/python3.10/site-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


KeyboardInterrupt: 

# 合并加载模型

In [27]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
from peft import PeftModel

mode_path = './autodl-tmp/qwen/Qwen2___5-7B-Instruct/'
lora_path = './output/mes_lora/checkpoint-100' # 这里改称你的 lora 输出对应 checkpoint 地址

# 加载tokenizer
tokenizer = AutoTokenizer.from_pretrained(mode_path, trust_remote_code=True)

# 加载模型
model = AutoModelForCausalLM.from_pretrained(mode_path, device_map="auto",torch_dtype=torch.bfloat16, trust_remote_code=True).eval()

# 加载lora权重
model = PeftModel.from_pretrained(model, model_id=lora_path)

prompt = "某批次B220的良率为85%，请分析影响因素并给出优化措施。"
inputs = tokenizer.apply_chat_template([{"role": "system", "content": "说明状况，分析什么设备什么原因，并给出优化建议"},{"role": "user", "content": prompt}],
                                       add_generation_prompt=True,
                                       tokenize=True,
                                       return_tensors="pt",
                                       return_dict=True
                                    #  ).to('cuda')
                                       ).to('cpu')


gen_kwargs = {"max_length": 2500, "do_sample": True, "top_k": 1}
with torch.no_grad():
    outputs = model.generate(**inputs, **gen_kwargs)
    outputs = outputs[:, inputs['input_ids'].shape[1]:]
    print(tokenizer.decode(outputs[0], skip_special_tokens=True))

loading file vocab.json
loading file merges.txt
loading file tokenizer.json
loading file added_tokens.json
loading file special_tokens_map.json
loading file tokenizer_config.json
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
loading configuration file ./autodl-tmp/qwen/Qwen2___5-7B-Instruct/config.json
Model config Qwen2Config {
  "_name_or_path": "./autodl-tmp/qwen/Qwen2___5-7B-Instruct/",
  "architectures": [
    "Qwen2ForCausalLM"
  ],
  "attention_dropout": 0.0,
  "bos_token_id": 151643,
  "eos_token_id": 151645,
  "hidden_act": "silu",
  "hidden_size": 3584,
  "initializer_range": 0.02,
  "intermediate_size": 18944,
  "max_position_embeddings": 32768,
  "max_window_layers": 28,
  "model_type": "qwen2",
  "num_attention_heads": 28,
  "num_hidden_layers": 28,
  "num_key_value_heads": 4,
  "rms_norm_eps": 1e-06,
  "rope_theta": 1000000.0,
  "sliding_window": null,
  "tie_word_embeddings": false,
  "torch_dtype": 

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

All model checkpoint weights were used when initializing Qwen2ForCausalLM.

All the weights of Qwen2ForCausalLM were initialized from the model checkpoint at ./autodl-tmp/qwen/Qwen2___5-7B-Instruct/.
If your task is similar to the task the model of the checkpoint was trained on, you can already use Qwen2ForCausalLM for predictions without further training.
loading configuration file ./autodl-tmp/qwen/Qwen2___5-7B-Instruct/generation_config.json
Generate config GenerationConfig {
  "bos_token_id": 151643,
  "do_sample": true,
  "eos_token_id": [
    151645,
    151643
  ],
  "pad_token_id": 151643,
  "repetition_penalty": 1.05,
  "temperature": 0.7,
  "top_k": 20,
  "top_p": 0.8
}



良率偏低可能由于温度控制不稳定或设备参数漂移，建议强化质检及工艺巡检机制。
